In [6]:
import open3d as o3d
import numpy as np
from sympy import*

In [100]:
def Ry_sym(theta):
    ct = cos(theta)
    st = sin(theta)
    R = Matrix([[ct, 0.0, st], [0.0, 1.0, 0.0], [-st, 0, ct]])
    return R

def Rx_sym(theta):
    ct = cos(theta)
    st = sin(theta)
    R = Matrix([[1.0, 0.0, 0.0], [0.0, ct, -st], [0.0, st, ct]])
    return R

def skew(v):
    return Matrix([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
                    
def exp3(omega, theta):
    omega = skew(omega)
    R = eye(3) + sin(theta) * omega + (1 - cos(theta)) * omega * omega
    return R

def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3,:3] = exp3(twist[:3], theta)
    T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
              (theta-sin(theta)) * omega * omega) * v
    return T

In [101]:
M1 = Matrix([[0, 1, 0, 0],
             [1, 0, 0, 0],
             [0, 0, -1, 200],
             [0, 0, 0, 1]])

M2 = Matrix([[1, 0, 0, 25],
             [0, 0, 1, 0],
             [0, -1, 0, 400],
             [0, 0, 0, 1]])

M3 = Matrix([[-1, 0, 0, 455+25],
             [0, 0, 1, 0],
             [0, 1, 0, 400],
             [0, 0, 0, 1]])

M4 = Matrix([[0, 0, -1, 455+25+420],
             [0, 1, 0, 0],
             [1, 0, 0, 400],
             [0, 0, 0, 1]])
M_list = np.array([M1,M2,M3,M4], dtype=float)

S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,25])
S4 = np.array([-1,0,0,0,-890,0])
S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
Slist[:, :4]


array([[   0,    0,    0,   -1],
       [   0,    1,    1,    0],
       [  -1,    0,    0,    0],
       [   0, -400, -855,    0],
       [   0,    0,    0, -890],
       [   0,   25,   25,    0]])

In [113]:
class RobotObject:
    robotObjects = [o3d.geometry.TriangleMesh.create_coordinate_frame(size=75)] #List of all o3d objects, initialized with {s}-frame

    def __init__(self, Mlist, link_orient = 'z'):
        self.Mlist = Mlist
        self.num_joints = len(Mlist)
        self.num_links = len(Mlist)
        self.length_links = self.findLinkLengths(Mlist)
        self.link_orient = ['z']*(self.num_links+1) if link_orient == 'z' else link_orient

        self.joints = []
        self.links = []
        self.make_robot_objects()
        self.transform(Mlist) #Transforms all objects to zero-config

#Calculates link lengths based on M
    def findLinkLengths(self, Mlist):
        linkLengths = np.zeros(self.num_links)
        
        for i in range(self.num_links):
            p = Mlist[i][:3,3]
            p_pre = np.array([0,0,0]) if i==0 else Mlist[i-1][:3,3] 
            linkLengths[i] = np.linalg.norm(p_pre-p)
        return linkLengths

#creates all o3d-objects of the robot in {s}
    def make_robot_objects(self): 
        for i in range(self.num_joints):
            self.joints.append(Joint())
        for i in range(self.num_links):
            self.links.append(Link(self.length_links[i], self.link_orient[i]))
    
    def theta_transform(self, Slist, thetas):
        T_list = []  # List with T01,T02,T03...
        T = np.eye(4)
        for i in range(len(thetas)):
            T = T @ exp6(Slist[:, i], thetas[i]) @ self.Mlist[i]
            T_list.append(T)
        self.transform(T_list)
        return


#Moves all objects from {s} to config given by T_list
    def transform(self, T_list):
        for i, J in enumerate(self.joints):
            J.transform(T_list[i])
        for i, L in enumerate(self.links):
            L.transform(T_list[i])

    def draw_robot(self):
        o3d.visualization.draw_geometries(self.robotObjects)

#_____________Joint Class______________#
class Joint(RobotObject):

    def __init__(self):
        self.joint = o3d.geometry.TriangleMesh.create_cylinder(
            radius=10, height=30)
        self.coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=25)
        self.update_mesh_list()
        self.set_colour()

    def update_mesh_list(self):
        self.robotObjects.append(self.joint)
        self.robotObjects.append(self.coord)

    def set_colour(self, colour=[0, 1, 0]):
        self.joint.paint_uniform_color(colour)

    def transform(self, T):
        self.joint = self.joint.transform(T)
        self.coord = self.coord.transform(T)

#____________Link Class_________________#
class Link(RobotObject):

    def __init__(self, lenght, orient):
        self.lenght = lenght
        
        if (orient == 'x'): ##Defines link direction
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).rotate(Ry_sym(np.pi/2)).translate(np.array([self.lenght/2, 0, 0]))
        elif (orient == 'y'):
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).rotate(Rx_sym(-np.pi/2)).translate(np.array([0, self.lenght/2, 0]))
        elif (orient == 'z'):
            self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=1, height=self.lenght).translate(np.array([0, 0, self.lenght/2]))
        
        self.update_mesh_list()
        self.set_colour()

    def update_mesh_list(self):
        self.robotObjects.append(self.link)

    def set_colour(self, colour=[0, 0, 1]):
        self.link.paint_uniform_color(colour)

    def transform(self, T):
        self.link = self.link.transform(T)

In [115]:
KUK = RobotObject( M_list, ['z', 'y', 'x', 'z'])
KUK.theta_transform(Slist, [-1.1416, -1.3258,  1.3102, -1.5708])
KUK.draw_robot()

In [82]:
p = M_list[2][:3,3]
p_pre = M_list[1][:3,3] 
linkLength = np.linalg.norm(p-p_pre)
KUK.length_links

array([200.        , 201.55644371, 455.        , 420.        ])